<a href="https://colab.research.google.com/github/Vignesh-P-C/fake-news-detection-transformers/blob/main/notebooks_05_first_training_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Training Run (Sanity Check)

**Project:** Fake News Detection using Transformers  
**Goal:** Verify end-to-end training works and loss decreases


In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)

MODEL_NAME = "distilbert-base-uncased"
MAX_LENGTH = 256

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
fake_df = pd.read_csv(
    "Fake.csv",
    engine="python",
    sep=",",
    quotechar='"',
    escapechar="\\",
    on_bad_lines="skip"
)

true_df = pd.read_csv(
    "True.csv",
    engine="python",
    sep=",",
    quotechar='"',
    escapechar="\\",
    on_bad_lines="skip"
)

fake_df["label"] = 0
true_df["label"] = 1

df = pd.concat([fake_df, true_df], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# FORCE correct types
texts = df["text"].astype(str).tolist()
labels = df["label"].tolist()

In [ ]:
type(texts), type(texts[0])

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

In [ ]:
assert isinstance(train_texts, list)
assert isinstance(train_texts[0], str)

In [ ]:
def tokenize_texts(texts):
    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH
    )

train_encodings = tokenize_texts(train_texts)
val_encodings = tokenize_texts(val_texts)


In [ ]:
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = NewsDataset(train_encodings, train_labels)
val_dataset = NewsDataset(val_encodings, val_labels)

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=100,
    save_strategy="no",
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()